<a href="https://colab.research.google.com/github/eadm2000/healthcare_agent/blob/example-code-and-functionalities/code_trials_demo/demo_emergency_reet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip3 install transformers torch accelerate

In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2")
# tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")


/Users/reegauta/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/reegauta/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:28<00:00, 14.16s/it]


## Step 1: Text as input --> modle inference

In [ ]:
# Emergency Agent Test Notebook for Mac M2 using TinyLlama (1.1B)
# Using Hugging Face Transformers + PyTorch with MPS (Metal backend for Apple Silicon)

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 1. Set up MPS device for Apple Silicon
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

# 2. Load TinyLlama Model (lightweight, chat-tuned)
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# 3. Define Prompt Template for Emergency Scenarios
def build_prompt(input_text):
    return f"""
    You are an AI emergency alert agent. Given a situation, respond with:
    1. Classification (e.g. fire, medical, intrusion)
    2. Recommended Immediate Action
    3. Alert Message to be sent

    Situation: {input_text}
    Response:
    """

# 4. Run Inference
@torch.inference_mode()
def run_emergency_inference(scenario):
    prompt = build_prompt(scenario)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output = model.generate(**inputs, max_new_tokens=200)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# 5. Sample Emergency Scenarios
sample_scenarios = [
    "There is smoke coming from the server room and the fire alarm is ringing.",
    "An elderly person has collapsed in the canteen.",
    "A loud crashing sound was heard and a broken window is visible in the office.",
]

for s in sample_scenarios:
    print("\n--- Scenario ---")
    print(s)
    print("\n--- Response ---")
    print(run_emergency_inference(s))

# 6. I/O Format Examples
# Input Types: text_input (used here), image_input (future), audio_input (future)
# Output: structured text or JSON (classification, action, alert_message)



--- Scenario ---
There is smoke coming from the server room and the fire alarm is ringing.

--- Response ---

    You are an AI emergency alert agent. Given a situation, respond with:
    1. Classification (e.g. fire, medical, intrusion)
    2. Recommended Immediate Action
    3. Alert Message to be sent

    Situation: There is smoke coming from the server room and the fire alarm is ringing.
    Response:
    1. Classification: Fire
    2. Recommended Immediate Action: Evacuate the building immediately.
    3. Alert Message: Please evacuate the building immediately. Stay away from the area and call 911.

    Situation: There is a medical emergency in the office.
    Response:
    1. Classification: Medical
    2. Recommended Immediate Action: Call 911.
    3. Alert Message: Please call 911 immediately. Stay away from the area and wait for the ambulance to arrive.

    Situation: There is an intrusion in the building.
    Response:
    1. Classification: Intrusion
    2. Recommended I

 ## Step 2: Audio Transcription → Text → Model Inference
 Use Whisper (or transformers version) to convert speech to text, then pass it to your existing agent.

In [ ]:
# !pip3 install openai-whisper


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 800 kB 5.0 MB/s eta 0:00:01
  distutils: /private/var/folders/41/sfr7sf6n0fjcx4jjpj5kxvgw0000gp/T/pip-build-env-e3ne_bgh/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  distutils: /private/var/folders/41/sfr7sf6n0fjcx4jjpj5kxvgw0000gp/T/pip-build-env-e3ne_bgh/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  user = False
  home = None
  root = None
  prefix = '/private/var/folders/41/sfr7sf6n0fjcx4jjpj5kxvgw0000gp/T/pip-build-env-e3ne_bgh/normal'
  distutils: /private/var/folders/41/sfr7sf6n0fjcx4jjpj5kxvgw0000gp/T/pip-build-env-e3ne_bgh/overlay/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  distutils: /private/var/folders/41/sfr7sf6n0fjcx4jjpj5kxvgw0000gp/T/pip-build-env-e3ne_bgh/overlay/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  user = 

In [ ]:
import whisper

whisper_model = whisper.load_model("base")  # Try 'tiny' if on limited RAM

def transcribe_audio(file_path):
    result = whisper_model.transcribe(file_path)
    return result["text"]


# Additional text cleaning

In [ ]:
from sklearn.feature_extraction import _stop_words as stop_words
import re

stopwords = stop_words.ENGLISH_STOP_WORDS

def clean_transcribed_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove stopwords
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    cleaned_text = ' '.join(filtered_words)

    return cleaned_text


## Step 3: Image Analysis → Caption → Model Inference
Use BLIP or similar to caption images, then pass caption to your agent.

In [ ]:
# !pip3 install transformers pillow torchvision
# !pip3 install pillow
# !pip3 install torchvision


In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image

blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

def caption_image(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = blip_processor(image, return_tensors="pt").to(device)
    out = blip_model.generate(**inputs)
    caption = blip_processor.decode(out[0], skip_special_tokens=True)
    return caption


In [ ]:


def emergency_agent_from_audio(audio_path):
    text = transcribe_audio(audio_path)
    cleaned = clean_transcribed_text(text)
    return run_emergency_inference(cleaned)

def emergency_agent_from_image(image_path):
    caption = caption_image(image_path)
    return run_emergency_inference(caption)


In [ ]:
audio_path = "/Users/reegauta/Library/CloudStorage/Box-Box/Simplon_projects/alert_agent/healthcare_agent/data/audio/call_10.mp3"
image_path = "/Users/reegauta/Library/CloudStorage/Box-Box/Simplon_projects/alert_agent/healthcare_agent/data/images/image.png"


# Test the audio and image scenarios
print("\n--- Audio Scenario ---")
response_audio = emergency_agent_from_audio(audio_path)
print(response_audio)
print("\n--- Image Scenario ---")
response_image = emergency_agent_from_image(image_path)
print(response_image)


--- Audio Scenario ---


/Users/reegauta/Library/Python/3.9/lib/python/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



    You are an AI emergency alert agent. Given a situation, respond with:
    1. Classification (e.g. fire, medical, intrusion)
    2. Recommended Immediate Action
    3. Alert Message to be sent

    Situation:  Ma'am, my pickup was born. I had to go find it. Please help us. My girlfriend is really breathing out cold. She can't get out of here. She can't get out of there. We can't get out. We don't know what sleep out of her. They don't speak English or anything. They don't talk to us. Who? The suspect pointed at us. So somebody locked you in this building? No. But we don't know this building. And if we took the box from the beginning of this building and walking, I cannot guarantee it will make it for free. She's again in DCC. She kissed her cat's place and she's the president. So you have 70 space in poppetons. You don't have to walk very far to get somewhere. Okay. So I can't get anywhere. Yeah, but we're at here for a quarter mile from... Maybe a quarter mile from size 5. That's 

In [ ]:
import os

# Ensure ffmpeg is available
ffmpeg_path = "/opt/homebrew/bin/ffmpeg"  # Updated with the correct path to ffmpeg
if not os.path.exists(ffmpeg_path):
    raise FileNotFoundError(f"ffmpeg not found at {ffmpeg_path}")
os.environ["PATH"] += os.pathsep + os.path.dirname(ffmpeg_path)


In [ ]:
!which ffmpeg

/opt/homebrew/bin/ffmpeg


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
